In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
URL = "https://somon.tj/nedvizhimost/prodazha-kvartir/"
page = requests.get(URL)
soup = BeautifulSoup(page.content,'html.parser')
page_list = soup.findAll("a",class_="page-number js-page-filter")
count_pages = int(str(page_list[-1]).split(">")[1][:str(page_list[-1]).split(">")[1].index("<")])+1

In [ ]:
URL = 'https://somon.tj/nedvizhimost/prodazha-kvartir/?page='
appartment_links = []

for i in range(1,count_pages):
    page = requests.get(URL+str(i))
    soup = BeautifulSoup(page.content,'html.parser')
    result = soup.find("ul", class_="list-simple__output js-list-simple__output")
    all_links = result.find_all("a", class_ = "mask")
    [appartment_links.append(str(i).split(" ")[2][6:-2]) for i in all_links]
    

In [ ]:
URL = 'https://somon.tj'
df_main = pd.DataFrame(columns=['city', 'time', 'announcment_num', 'floor', 'area', 'room_count', 'district', 'description', 'discount', 'price'])
for i in range(len(appartment_links)):
    page = requests.get(URL + appartment_links[i])
    print(URL + appartment_links[i])
    soup = BeautifulSoup(page.content,'html.parser')

    try:
        city = soup.find("a", class_ = "announcement__location js-open-announcement-location").find('span').get_text()
    except:
        city = None
    
    time = soup.find("span", class_ = "date-meta").get_text()[14:]
    announcment_num = int(soup.find("span", class_ = "number-announcement").get_text()[18:])
    try:
        floor = str(soup.findAll('a', class_ = 'value-chars')[2]).split('>')[1].split("<")[0]
    except:
        floor = None
    
    try:
        building_type = str(soup.findAll('a', class_ = 'value-chars')[1]).split('>')[1].split("<")[0]
    except:
        building_type = None
    
    try:
        area = str(soup.findAll('a', class_ = 'value-chars')[0]).split('>')[1].split("<")[0]
    except:
        area = None
    
    try:
        room_count = soup.find('h1', 'title-announcement').get_text().replace(" ", '').replace("\n", '').split("-")[0]
    except:
        room_count = None
    
    try:
        district = str(soup.find('span', class_ = 'value-chars').get_text().replace(" ", "").replace("\n", ''))
    except:
        district = None
    
    try:
        description = str(soup.find('div', class_='announcement-description').find("p")).split('>')[1].split("<")[0]
    except:
        description = None
    
    try:
        discount = soup.find('div', class_ ='announcement-price__cost').get_text().replace(" ", '').replace("\n", '').split(".")[1]
    except:
        discount = None
    price = int(soup.find('div', class_ ='announcement-price__cost').get_text().replace(" ", '').replace("\n", '').split(".")[0][:-1])
    dt = {"city":city, "time":time, "announcment_num":announcment_num, "floor":floor, "area":area, "room_count":room_count, "district":district, "description":description, "discount":discount, "price":price}
    df_main = df_main.append(dt, ignore_index=True)

In [ ]:
df_main.to_csv("data.csv")